<h1><center>Cancer treatment prediction </center></h1>

kaggle competitions download -c msk-redefining-cancer-treatment

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:lightblue; border:0; color:white' role="tab" aria-controls="home"><center>Quick navigation</center></h2>

* [0. Installation of libraries](#0)
* [1. Basic Data Overview](#1)
* [2. Analysis of data](#2)
* [3. EDA](#3)
* [4. Classification models](#4)


<a id="0"></a>
<h2 style='background:lightblue; border:0; color:white'><center>0. 📚 Installation of libraries</center><h2>

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from  sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing  import LabelEncoder
from sklearn import linear_model 
from sklearn import tree 
from sklearn import ensemble 
from sklearn import metrics 
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split 
from sklearn.feature_selection import SelectKBest, f_classif
import plotly 
import plotly.express as px
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

import gensim

import scikitplot.plotters as skplt

import nltk

from xgboost import XGBClassifier

import os

from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.optimizers import Adam

import zipfile

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


<a id="1"></a>
<h2 style='background:lightblue; border:0; color:white'><center>1. Basic Data Overview</center><h2>

develop a Machine Learning algorithm that, using this knowledge base as a baseline, automatically classifies genetic variations.

**training_variants** - a comma separated file containing the description of the genetic mutations used for training. Fields are ID (the id of the row used to link the mutation to the clinical evidence), Gene (the gene where this genetic mutation is located), Variation (the aminoacid change for this mutations), Class (1-9 the class this genetic mutation has been classified on)

**training_text** - a double pipe (||) delimited file that contains the clinical evidence (text) used to classify genetic mutations. Fields are ID (the id of the row used to link the clinical evidence to the genetic mutation), Text (the clinical evidence used to classify the genetic mutation)

**test_variants** - a comma separated file containing the description of the genetic mutations used for training. Fields are ID (the id of the row used to link the mutation to the clinical evidence), Gene (the gene where this genetic mutation is located), Variation (the aminoacid change for this mutations)

**test_text** - a double pipe (||) delimited file that contains the clinical evidence (text) used to classify genetic mutations. Fields are ID (the id of the row used to link the clinical evidence to the genetic mutation), Text (the clinical evidence used to classify the genetic mutation)

submissionSample - a sample submission file:

For each ID in the test set, you must predict a probability for each of the different classes a genetic mutation can be classified on. The file should contain a header and have the following format:

ID,class1,class2,class3,class4,class5,class6,class7,class8,class9

In [2]:
train_text = pd.read_csv('/kaggle/input/unzip-files/training_text',
                         sep='\|\|', 
                         header=None, 
                         skiprows=1, 
                         names=["ID","Text"])
display(train_text.info(),
       train_text.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3321 entries, 0 to 3320
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      3321 non-null   int64 
 1   Text    3316 non-null   object
dtypes: int64(1), object(1)
memory usage: 52.0+ KB


None

,ID,Text
0,0,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,Abstract Background Non-small cell lung canc...
2,2,Abstract Background Non-small cell lung canc...
3,3,Recent evidence has demonstrated that acquired...
4,4,Oncogenic mutations in the monomeric Casitas B...


In [3]:
train_var = pd.read_csv('/kaggle/input/unzip-files/training_variants')
display(train_var.info(),
       train_var.head(),
       train_var.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3321 entries, 0 to 3320
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         3321 non-null   int64 
 1   Gene       3321 non-null   object
 2   Variation  3321 non-null   object
 3   Class      3321 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 103.9+ KB


None

,ID,Gene,Variation,Class
0,0,FAM58A,Truncating Mutations,1
1,1,CBL,W802*,2
2,2,CBL,Q249E,2
3,3,CBL,N454D,3
4,4,CBL,L399V,4


,ID,Class
count,3321.000000,3321.000000
mean,1660.000000,4.365854
std,958.834449,2.309781
min,0.000000,1.000000
25%,830.000000,2.000000
50%,1660.000000,4.000000
75%,2490.000000,7.000000
max,3320.000000,9.000000


In [4]:
train = train_var.merge(train_text, how='left', on='ID')
display(train.info(),
       train.head()
       )

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3321 entries, 0 to 3320
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         3321 non-null   int64 
 1   Gene       3321 non-null   object
 2   Variation  3321 non-null   object
 3   Class      3321 non-null   int64 
 4   Text       3316 non-null   object
dtypes: int64(2), object(3)
memory usage: 155.7+ KB


None

,ID,Gene,Variation,Class,Text
0,0,FAM58A,Truncating Mutations,1,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,CBL,W802*,2,Abstract Background Non-small cell lung canc...
2,2,CBL,Q249E,2,Abstract Background Non-small cell lung canc...
3,3,CBL,N454D,3,Recent evidence has demonstrated that acquired...
4,4,CBL,L399V,4,Oncogenic mutations in the monomeric Casitas B...


In [5]:
train.dropna(axis=0, inplace=True)

In [6]:
fig = px.pie(
    values=train['Class'].value_counts(normalize=True).values,
    names=train['Class'].value_counts(normalize=True).index,
    title='Class Distribution'
)

fig.show()

In [7]:
from datasets import Dataset,DatasetDict
train_ds = Dataset.from_pandas(train)

In [8]:
train['text len']= train['Text'].apply(lambda x: len(x.split()))
train['text len'].max()

76782

Tokenization

In [19]:
# for long text
from transformers import LongformerTokenizer, LongformerForSequenceClassification
model_name = "allenai/longformer-base-4096"
model = LongformerForSequenceClassification.from_pretrained(model_name, num_labels=9)
tokenizer = LongformerTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    # Tokenize the text and return the tokenized inputs as a dictionary
    inputs = tokenizer(examples['Text'], padding=True, truncation=True, max_length=100000)
    return inputs

train_tok = train_ds.map(tokenize_function, batched=True)
train_tok = train_tok.rename_columns({'Text':'labels'})
# Assuming tok_dataset is a tokenized dataset
sample = train_tok[0]

# Check if 'labels' key is present and if its value is a flat list of integers
if 'labels' in sample and isinstance(sample['labels'], list) and all(isinstance(label, int) for label in sample['labels']):
    print("Labels are flattened.")
else:
    print("Labels are not flattened.")

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias'

  0%|          | 0/4 [00:00<?, ?ba/s]

Labels are not flattened.


Embedding

In [18]:
#pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [20]:
from gensim.models import KeyedVectors
import numpy as np

# Load pre-trained GloVe embeddings
glove_model = KeyedVectors.load_word2vec_format('path/to/glove.6B.50d.txt', binary=False)

# Assuming train_tok has a 'tokens' column containing tokenized text
tokens_column = train_tok['labels']

# Define the dimension of your GloVe embeddings (50 in this case)
embedding_dim = 50

# Create an array to store the embeddings
embeddings_matrix = np.zeros((len(tokens_column), len(tokens_column[0]), embedding_dim))

# Iterate through each token in each row and assign the GloVe embedding
for i, tokens in enumerate(tokens_column):
    for j, token in enumerate(tokens):
        try:
            embeddings_matrix[i, j, :] = glove_model[token]
        except KeyError:
            # Handle out-of-vocabulary tokens (initialize randomly or with zeros)
            embeddings_matrix[i, j, :] = np.random.normal(size=(embedding_dim,))

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/glove.6B.50d.txt'

logistic regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Flatten the embeddings_matrix
flattened_embeddings = embeddings_matrix.reshape((len(tokens_column), -1))

# Assuming train_tok has a 'classification_labels' column containing classification targets
classification_labels = train_tok['Class']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(flattened_embeddings, classification_labels, test_size=0.2, random_state=42)

# Initialize and train a classification model (e.g., Logistic Regression)
classification_model = LogisticRegression()
classification_model.fit(X_train, y_train)

# Make predictions on the test set
predictions = classification_model.predict(X_test)

# Evaluate the classification model
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)


In [ ]:

dds = train_tok.train_test_split(0.25, seed=42)

support vector machines

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Flatten the embeddings_matrix
flattened_embeddings = embeddings_matrix.reshape((len(tokens_column), -1))

# Assuming train_tok has a 'classification_labels' column containing classification targets
classification_labels = train_tok['Class']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(flattened_embeddings, classification_labels, test_size=0.2, random_state=42)

# Initialize and train an SVM classifier
svm_classifier = SVC(kernel='linear')  # You can choose different kernels (linear, rbf, poly, etc.)
svm_classifier.fit(X_train, y_train)

# Make predictions on the test set
predictions = svm_classifier.predict(X_test)

# Evaluate the SVM classifier
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)


random forests

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Flatten the embeddings_matrix
flattened_embeddings = embeddings_matrix.reshape((len(tokens_column), -1))

# Assuming train_tok has a 'classification_labels' column containing classification targets
classification_labels = train_tok['Class']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(flattened_embeddings, classification_labels, test_size=0.2, random_state=42)

# Initialize and train a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust the number of trees (n_estimators)
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
predictions = rf_classifier.predict(X_test)

# Evaluate the Random Forest classifier
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)


transformers (e.g., BERT, GPT)

<a id="3"></a>
<h2 style='background:lightblue; border:0; color:white'><center>3. Arguments</center><h2>

In [9]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
from transformers import TrainingArguments,Trainer


In [10]:
batch_size = 32
epochs = 4
lr = 8e-5

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [11]:
#fine-tune
args = TrainingArguments(
    output_dir='./output',
    logging_dir='./logs',
    
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size*2,
    
    num_train_epochs=epochs,
    evaluation_strategy="epoch", #Specifies after what to evaluate the model during training:"epoch", "steps","no" 
    
    learning_rate=lr,
    lr_scheduler_type='cosine', #a cosine annealing scheduler: The learning rate starts high and gradually decreases in a cosine pattern, 
                                #helping the model converge more smoothly.
                                #'linear', 'constant', or custom learning rate schedulers.
    warmup_ratio=0.1, #Warmup is a technique where the learning rate starts very low and gradually increases for a certain number of steps. 
                      #It helps stabilize training at the beginning.
                      #The warmup ratio is the fraction of total training steps during which the warmup is applied.
    weight_decay=0.01, # regularization technique that penalizes large weights in the model.
                       #This parameter specifies the strength of the weight decay. 
                       #A higher value means stronger regularization.
    
    logging_steps=500,#Log training information every logging_steps steps
    do_train=True,
    fp16=False,  #Whether to use mixed-precision training (16-bit floating-point) to reduce memory usage.
    report_to='none'
)


Before tokenization, there are several important considerations to keep in mind:

**Tokenization Method:**

Choose an appropriate tokenizer based on your task and the model you intend to use. Different models may have different tokenization requirements.

**Special Tokens:**

Be aware of any special tokens that your model requires, such as [CLS] and [SEP] tokens for many sequence classification tasks.

**Maximum Sequence Length:**

Check the maximum sequence length supported by your model. If your input sequences are longer than the model's maximum, you may need to truncate or split them.

**Padding and Truncation:**

Decide on a strategy for handling sequences that are shorter or longer than the desired length. Padding is often used for shorter sequences, and truncation may be necessary for longer sequences.

**Attention Masks:**

Understand the concept of attention masks. They are used to indicate which tokens are actual input and which are padding tokens. Ensure that your tokenization process generates attention masks correctly.

**Special Tokens for Labels:**

In some tasks, you might need to include special tokens to represent labels or targets. Make sure you understand the specific requirements for your task.

**Token Indexing:**

Verify how tokens are indexed. Some tokenizers return token IDs directly, while others return tokens that need to be converted to IDs.

**Flattening Nested Labels:**

If your labels have nested structures, like a list of lists, you may need to flatten them to a single list of integers before tokenization.

**Vocabulary Size:**

Be aware of the vocabulary size of your tokenizer. If your text contains out-of-vocabulary words, you may need to handle them appropriately.

**Encoding Formats:**

Understand the encoding formats your model accepts (e.g., PyTorch tensors, TensorFlow tensors). Ensure that your tokenization process outputs the appropriate format.

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias'

  0%|          | 0/4 [00:00<?, ?ba/s]

In [15]:
train_tok = train_tok.rename_columns({'Text':'labels'})
dds = train_tok.train_test_split(0.25, seed=42)

Labels are not flattened.


In [17]:
# Training
trainer = Trainer(
    model,
    args,
    train_dataset=dds['train'],
    eval_dataset=dds['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

# Save the fine-tuned model
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`labels` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

<a id="3"></a>
<h2 style='background:lightblue; border:0; color:white'><center>3. Hugging Face and Bigscience</center><h2>

In [ ]:
from transformers import T5Tokenizer, AutoModelForSeq2SeqLM

# Update tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")
def tokenize_function(examples):
    # Tokenize the text
    inputs = tokenizer(examples['Text'], return_tensors='pt', padding=True, truncation=True)
    
    # Ensure 'labels' is converted to a list
    labels = examples['Text']
    if isinstance(labels, torch.Tensor):
        labels = labels.tolist()
    
    # Return a dictionary with inputs and labels as lists
    return {
        'input_ids': inputs['input_ids'].tolist(),
        'attention_mask': inputs['attention_mask'].tolist(),
        'labels': labels,
    }
tok_train = train_ds.map(tokenize_function, batched=True)
tok_train = tok_train.rename_columns({'Text':'labels'})
dds = tok_train.train_test_split(0.25, seed=42)

# Fine-tune model configuration and training arguments
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0_3B")

# Training
trainer = Trainer(
    model,
    args,
    train_dataset=dds['train'],
    eval_dataset=dds['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

# Save the fine-tuned model
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')


<a id="4"></a>
<h2 style='background:lightblue; border:0; color:white'><center>4. DeBERTa-v2 model architecture</center><h2>

In [ ]:
from transformers import DebertaV2TokenizerFast, DebertaV2ForSequenceClassification
# Load pre-trained model and tokenizer
model_nm = 'microsoft/deberta-v2-xlarge'
model = DebertaV2ForSequenceClassification.from_pretrained(model_nm, num_labels=9)
tokenizer = DebertaV2TokenizerFast.from_pretrained(model_nm)

#labels feature has excessive nesting, 
#you need to ensure that it is flattened to a list of integers before tokenization
def tokenize_function(examples):
    # Flatten the nested list of labels to a flat list of integers
    flat_labels = [label for sublist in examples['Text'] for label in sublist]
    # Tokenize the text and return the tokenized inputs
    return tokenizer(examples['Text'], labels=flat_labels, padding=True, truncation=True, return_tensors='pt')


tok_train = train_ds.map(tokenize_function, batched=True)
tok_train = tok_train.rename_columns({'Text':'labels'})
dds = tok_train.train_test_split(0.25, seed=42)
dds

In [ ]:
trainer = Trainer(model, 
                  args, 
                  train_dataset=dds['train'], 
                  eval_dataset=dds['test'],
                  tokenizer=tokenizer, 
                  compute_metrics=compute_metrics)

trainer.train()

# Save the fine-tuned model
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')

In [ ]:
test_text = pd.read_csv('/kaggle/input/unzip-files/test_text',
                         sep='\|\|', 
                         header=None, 
                         skiprows=1, 
                         names=["ID","Text"])
test_var = pd.read_csv('/kaggle/input/unzip-files/test_variants')
test = test_var.merge(test_text, how='left', on='ID')
test.dropna(axis=0, inplace=True)

In [ ]:
test_ds = Dataset.from_pandas(test.map(tok_func, batched=True)
preds = trainer.predict(test_true_ds).predictions.astype(int)
from sklearn.metrics import classification_report

# Assuming you have true labels in test_true_ds
true_labels = test_true['Sentiment']

# Convert predicted probabilities to labels
pred_labels = preds.argmax(axis=1)

# Generate a classification report
class_report = classification_report(true_labels, pred_labels)

# Print or use the classification report as needed
print(class_report)

In [ ]:
test_true = pd.read_csv('/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv', encoding='ISO-8859-1')
test_true['Sentiment']=coder.fit_transform(test_true['Sentiment'])
display(test_true.info(),
       test_true.describe(),
       test_true.head())

test_true_ds = Dataset.from_pandas(test_true).map(tok_func, batched=True)
preds = trainer.predict(test_true_ds).predictions.astype(int)
from sklearn.metrics import classification_report

# Assuming you have true labels in test_true_ds
true_labels = test_true['Sentiment']

# Convert predicted probabilities to labels
pred_labels = preds.argmax(axis=1)

# Generate a classification report
class_report = classification_report(true_labels, pred_labels)

# Print or use the classification report as needed
print(class_report)

In [ ]:
df_test_txt = pd.read_csv('../input/test_text', sep='\|\|', header=None, skiprows=1, names=["ID","Text"])
df_test_txt.head()

In [ ]:
df_test_var = pd.read_csv('../input/test_variants')
df_test_var.head()

### Let's join them together

In [ ]:
df_train = pd.merge(df_train_var, df_train_txt, how='left', on='ID')
df_train.head()

In [ ]:
df_test = pd.merge(df_test_var, df_test_txt, how='left', on='ID')
df_test.head()

## Run preliminary statistics on loaded data

In [ ]:
df_train.describe(include='all')

In [ ]:
df_test.describe(include='all')

In [ ]:
df_train['Class'].value_counts().plot(kind="bar", rot=0)

### Classes seem very imbalanced

In [ ]:
# This cell reduces the training data for Kaggle limits. Remove this cell for real results.
df_train, _ = train_test_split(df_train, test_size=0.7, random_state=8, stratify=df_train['Class'])
df_train.shape

# The main task here is to predict the class of the mutation given the text in the literature. Our approach will then be to apply some common NLP techniques to transform the free text into features for an ML classifier and see which ones work best.

### Define a helper function to evaluate the effectiveness of transformed free text. We'll use a simple logistic regression with 3-fold stratified cross-validation for fast evaluation.

In [ ]:
def evaluate_features(X, y, clf=None):
    """General helper function for evaluating effectiveness of passed features in ML model
    
    Prints out Log loss, accuracy, and confusion matrix with 3-fold stratified cross-validation
    
    Args:
        X (array-like): Features array. Shape (n_samples, n_features)
        
        y (array-like): Labels array. Shape (n_samples,)
        
        clf: Classifier to use. If None, default Log reg is use.
    """
    if clf is None:
        clf = LogisticRegression()
    
    probas = cross_val_predict(clf, X, y, cv=StratifiedKFold(random_state=8), 
                              n_jobs=-1, method='predict_proba', verbose=2)
    pred_indices = np.argmax(probas, axis=1)
    classes = np.unique(y)
    preds = classes[pred_indices]
    print('Log loss: {}'.format(log_loss(y, probas)))
    print('Accuracy: {}'.format(accuracy_score(y, preds)))
    skplt.plot_confusion_matrix(y, preds)


Let's do a quick test of evaluate_features

In [ ]:
# Quick test of evaluate_features
from sklearn.datasets import load_iris
evaluate_features(*load_iris(True))

## Start with a simple baseline. Bag of words

In [ ]:
count_vectorizer = CountVectorizer(
    analyzer="word", tokenizer=nltk.word_tokenize,
    preprocessor=None, stop_words='english', max_features=None)    


In [ ]:
bag_of_words = count_vectorizer.fit_transform(df_train['Text'])

In [ ]:
len(count_vectorizer.get_feature_names())

#### 281586 unique words in corpus

In [ ]:
svd = TruncatedSVD(n_components=25, n_iter=25, random_state=12)
truncated_bag_of_words = svd.fit_transform(bag_of_words)

In [ ]:
evaluate_features(truncated_bag_of_words, df_train['Class'].values.ravel())

In [ ]:
evaluate_features(truncated_bag_of_words, df_train['Class'].values.ravel(), 
                  RandomForestClassifier(n_estimators=1000, max_depth=5, verbose=1))

### Bad results overall for the baseline

## Let's try TFIDF

In [ ]:
count_vectorizer = TfidfVectorizer(
    analyzer="word", tokenizer=nltk.word_tokenize,
    preprocessor=None, stop_words='english', max_features=None)    

tfidf = count_vectorizer.fit_transform(df_train['Text'])

len(count_vectorizer.get_feature_names())

In [ ]:
svd = TruncatedSVD(n_components=25, n_iter=25, random_state=12)
truncated_tfidf = svd.fit_transform(tfidf)

In [ ]:
evaluate_features(truncated_tfidf, df_train['Class'].values.ravel())

In [ ]:
evaluate_features(truncated_tfidf, df_train['Class'].values.ravel(), 
                  RandomForestClassifier(n_estimators=1000, max_depth=5, verbose=1))

In [ ]:
evaluate_features(tfidf, df_train['Class'].values.ravel(), 
                  SVC(kernel='linear', probability=True))

### A little better, but still bad. You can see from the confusion matrix that it's just classifying most samples into class 7.

### Also tried a linear SVM for features straight from TFIDF (did not go through Truncated SVD). Worse log loss but confusion matrix seems to show better balance among predicted classes.

_____

## This time, let's try the popular word2vec to get features

### Define helper function get_word2vec  and helper class MySentences for training word2vec on the corpus of texts. (or loading if already trained and saved to disk)

In [ ]:
class MySentences(object):
    """MySentences is a generator to produce a list of tokenized sentences 
    
    Takes a list of numpy arrays containing documents.
    
    Args:
        arrays: List of arrays, where each element in the array contains a document.
    """
    def __init__(self, *arrays):
        self.arrays = arrays
 
    def __iter__(self):
        for array in self.arrays:
            for document in array:
                for sent in nltk.sent_tokenize(document):
                    yield nltk.word_tokenize(sent)

def get_word2vec(sentences, location):
    """Returns trained word2vec
    
    Args:
        sentences: iterator for sentences
        
        location (str): Path to save/load word2vec
    """
    if os.path.exists(location):
        print('Found {}'.format(location))
        model = gensim.models.Word2Vec.load(location)
        return model
    
    print('{} not found. training model'.format(location))
    model = gensim.models.Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)
    print('Model done training. Saving to disk')
    model.save(location)
    return model

### Start training the word2vec model. Since word2vec training is unsupervised, you can use both training and test datasets.

If training has already been done, the function will just load the word2vec to disk so you don't need to retrain if rerunning the notebook

In [ ]:
w2vec = get_word2vec(
    MySentences(
        df_train['Text'].values, 
        #df_test['Text'].values  Commented for Kaggle limits
    ),
    'w2vmodel'
)

### Now that we have our word2vec model, how do we use it to transform each documents into a feature vector? In order to convert a document of multiple words into a single vector using our trained word2vec, we take the word2vec of all words in the document, then take its mean.

### We'll define a transformer (with sklearn interface) to convert a document into its corresponding vector

In [ ]:
class MyTokenizer:
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        transformed_X = []
        for document in X:
            tokenized_doc = []
            for sent in nltk.sent_tokenize(document):
                tokenized_doc += nltk.word_tokenize(sent)
            transformed_X.append(np.array(tokenized_doc))
        return np.array(transformed_X)
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.wv.syn0[0])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = MyTokenizer().fit_transform(X)
        
        return np.array([
            np.mean([self.word2vec.wv[w] for w in words if w in self.word2vec.wv]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
    def fit_transform(self, X, y=None):
        return self.transform(X)


In [ ]:
mean_embedding_vectorizer = MeanEmbeddingVectorizer(w2vec)
mean_embedded = mean_embedding_vectorizer.fit_transform(df_train['Text'])

In [ ]:
evaluate_features(mean_embedded, df_train['Class'].values.ravel())

In [ ]:
evaluate_features(mean_embedded, df_train['Class'].values.ravel(),
                  RandomForestClassifier(n_estimators=1000, max_depth=15, verbose=1))

In [ ]:
evaluate_features(mean_embedded, 
                  df_train['Class'].values.ravel(),
                  XGBClassifier(max_depth=4,
                                objective='multi:softprob',
                                learning_rate=0.03333,
                                )
                 )

### As expected, we get better results than TF-IDF. 

The results are still not very good though. One way to explain this is that there is a lot of information loss from just getting the mean of all word vectors of the document. This is roughly analogous to taking the entire document, summarizing it into one word, and using that word to classify the entire text.

## Let's try a quick and dirty LSTM in Keras to take into account the sequential nature of text

* We won't do any hyperparameter search 
* We'll go with 15 epochs, and save the model with the best validation loss after an epoch
* Max sequence length is cut down to a measly 2000 (longest text has 77000+ words), to shorten training time and prevent GPU OOM

Note: This takes about an hour to run on GPU

In [ ]:
# Use the Keras tokenizer
num_words = 2000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(df_train['Text'].values)

In [ ]:
# Pad the data 
X = tokenizer.texts_to_sequences(df_train['Text'].values)
X = pad_sequences(X, maxlen=2000)

In [ ]:
# Build out our simple LSTM
embed_dim = 128
lstm_out = 196

# Model saving callback
ckpt_callback = ModelCheckpoint('keras_model', 
                                 monitor='val_loss', 
                                 verbose=1, 
                                 save_best_only=True, 
                                 mode='auto')

model = Sequential()
model.add(Embedding(num_words, embed_dim, input_length = X.shape[1]))
model.add(LSTM(lstm_out, recurrent_dropout=0.2, dropout=0.2))
model.add(Dense(9,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['categorical_crossentropy'])
print(model.summary())

In [ ]:
Y = pd.get_dummies(df_train['Class']).values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42, stratify=Y)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)


In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs=8, batch_size=batch_size, validation_split=0.2, callbacks=[ckpt_callback])

In [ ]:
model = load_model('keras_model')

In [ ]:
probas = model.predict(X_test)

In [ ]:
pred_indices = np.argmax(probas, axis=1)
classes = np.array(range(1, 10))
preds = classes[pred_indices]
print('Log loss: {}'.format(log_loss(classes[np.argmax(Y_test, axis=1)], probas)))
print('Accuracy: {}'.format(accuracy_score(classes[np.argmax(Y_test, axis=1)], preds)))
skplt.plot_confusion_matrix(classes[np.argmax(Y_test, axis=1)], preds)


### The results of the quick LSTM are promising. 

On the first try with no hyperparameter search, 6th epoch, max sequence length cut down to a measly 2000 (longest text has 77000+ words), we get the best log loss so far of around 1.4. You can still see a bit of bias towards class 7 but the confusion matrix looks more balanced than anything we've seen so far.

### Further tuning of the LSTM will likely produce better results.

## So far, we've only used the text field to perform classification. But there is still the "Gene" and "Variation" fields.

Using only the Text field is a bit flawed. Looking closer at the statistics we calculated above, "training_text" actually has duplicates, and the duplicates have different classes. This is part of the challenge. A lot of papers are studies of 2 or more genes. It is our job to use the other fields to figure out which parts of the text are relevant for the particular Gene and Variation.

### Let's use a LabelEncoder to encode Gene and Variation and combine it with TFIDF

In [ ]:
gene_le = LabelEncoder()
gene_encoded = gene_le.fit_transform(df_train['Gene'].values.ravel()).reshape(-1, 1)
gene_encoded = gene_encoded / np.max(gene_encoded)

In [ ]:
variation_le = LabelEncoder()
variation_encoded = variation_le.fit_transform(df_train['Variation'].values.ravel()).reshape(-1, 1)
variation_encoded = variation_encoded / np.max(variation_encoded)

In [ ]:
evaluate_features(np.hstack((gene_encoded, variation_encoded, truncated_tfidf)), df_train['Class'])

In [ ]:
evaluate_features(np.hstack((gene_encoded, variation_encoded, truncated_tfidf)), df_train['Class'],
                  RandomForestClassifier(n_estimators=1000, max_depth=5, verbose=1))

### Barely any difference, let's try our  label encoded features with our word2vec features

In [ ]:
evaluate_features(np.hstack((gene_encoded, variation_encoded, mean_embedded)), df_train['Class'])

In [ ]:
evaluate_features(np.hstack((gene_encoded, variation_encoded, mean_embedded)), df_train['Class'],
                  RandomForestClassifier(n_estimators=1000, max_depth=5, verbose=1))

### Doesn't make a difference either. Let's try one-hot encoding + SVD the "Gene" and "Variation" features

In [ ]:
one_hot_gene = pd.get_dummies(df_train['Gene'])
svd = TruncatedSVD(n_components=25, n_iter=25, random_state=12)
truncated_one_hot_gene = svd.fit_transform(one_hot_gene.values)

In [ ]:
one_hot_variation = pd.get_dummies(df_train['Variation'])
svd = TruncatedSVD(n_components=25, n_iter=25, random_state=12)
truncated_one_hot_variation = svd.fit_transform(one_hot_variation.values)

### Truncated one hot encoding + TFIDF

In [ ]:
evaluate_features(np.hstack((truncated_one_hot_gene, truncated_one_hot_variation, truncated_tfidf)), df_train['Class'])

In [ ]:
evaluate_features(np.hstack((truncated_one_hot_gene, truncated_one_hot_variation, truncated_tfidf)), df_train['Class'],
                  RandomForestClassifier(n_estimators=1000, max_depth=5, verbose=1))


### Truncated one hot encoding + word2vec

In [ ]:
evaluate_features(np.hstack((truncated_one_hot_gene, truncated_one_hot_variation, mean_embedded)), df_train['Class'])

In [ ]:
evaluate_features(np.hstack((truncated_one_hot_gene, truncated_one_hot_variation, mean_embedded)), df_train['Class'],
                  RandomForestClassifier(n_estimators=1000, max_depth=5, verbose=1))

### Interestingly, performance is actually a bit worse than simple label encoding

## Before going into a summary of the insights we've discovered, let's generate some submissions from our best models and see how they fare in the public leaderboard

### We'll start by generating a submission from our word2vec model

In [ ]:
lr_w2vec = LogisticRegression()
lr_w2vec.fit(mean_embedded, df_train['Class'])

In [ ]:
mean_embedded_test = mean_embedding_vectorizer.transform(df_test['Text'])

In [ ]:
probas = lr_w2vec.predict_proba(mean_embedded_test)

In [ ]:
submission_df = pd.DataFrame(probas, columns=['class'+str(c+1) for c in range(9)])
submission_df['ID'] = df_test['ID']
submission_df.head()

In [ ]:
submission_df.to_csv('submission.csv', index=False)

### Test out XGB and SVC

In [ ]:
xgb_w2vec = XGBClassifier(max_depth=4,
                          objective='multi:softprob',
                          learning_rate=0.03333)
xgb_w2vec.fit(mean_embedded, df_train['Class'])
probas = xgb_w2vec.predict_proba(mean_embedded_test)
submission_df = pd.DataFrame(probas, columns=['class'+str(c+1) for c in range(9)])
submission_df['ID'] = df_test['ID']
submission_df.to_csv('submission.csv', index=False)

In [ ]:
svc_w2vec = SVC(kernel='linear', probability=True)
svc_w2vec.fit(mean_embedded, df_train['Class'])
probas = svc_w2vec.predict_proba(mean_embedded_test)
submission_df = pd.DataFrame(probas, columns=['class'+str(c+1) for c in range(9)])
submission_df['ID'] = df_test['ID']
submission_df.to_csv('submission.csv', index=False)

#### Public LB Score Log Reg: 1.032000

#### Public LB Score XGB: 0.96536

#### Public LB Score SVC: 0.97059

### Let's try our Keras model

In [ ]:
Xtest = tokenizer.texts_to_sequences(df_test['Text'].values)
Xtest = pad_sequences(Xtest, maxlen=2000)

In [ ]:
probas = model.predict(Xtest)

In [ ]:
submission_df = pd.DataFrame(probas, columns=['class'+str(c+1) for c in range(9)])
submission_df['ID'] = df_test['ID']
submission_df.head()

In [ ]:
submission_df.to_csv('submission.csv', index=False)

#### Public LB Score: 1.00234

## Summary

The aim of this notebook was to do some quick exploration of the dataset and apply some common ML techniques to the classification task. The metric to maximize is multiclass log loss.

A big part of the problem is to teach an ML model how to "read" medical literature and classify the given Gene and Variation into 1 out of 9 classes.

Thus, the first part of this notebook focused on applying common techniques to preprocess and vectorize free text and evaluate its effectiveness by running them through vanilla Logistic Regression and Random Forest.

The techniques used, from least effective to most effective, were:

* Bag of Words
* TF-IDF
* Word2Vec

Because the above approaches did not take into account the temporal patterns in free text, a quick LSTM was tried as well. This approach scored higher than the above without any tuning.

In the second part of the notebook, I added the "Gene" and "Variation" features next to the free text features. I tried both label encoding and one-hot encoding, however, the results did not show much improvement.

In the third part of the notebook, I generated submissions for both Word2Vec (multiple classifiers) and Keras LSTM and recorded the public leaderboard scores of each submission. The scores were better (around 1) but did not show the same relationships with each other as my own CV (they were mostly close to each other). This is a common occurrence in Kaggle competitions since the public leaderboard is scored on a smaller subset of the test data. Most Kagglers' advice is to ignore the public leaderboard and trust your own CV.

## Further things to try

This notebook's aim was mostly figuring out which techniques are worth exploring and was not intended to generate very competitive submissions. The following is a list of suggestions to try for further improvement.

* There are tons of other techniques for free text other than the ones I listed above. Make sure to explore other techniques such as Doc2Vec, DeepIR, and Word Mover's distance

* Focus more on capturing the relationship between "Gene" and "Variation" with the free text features. Since "Text" is sometimes duplicated (with different classes!), taking into account "Gene" and "Variation" is very important.

* Explore different deep learning architectures for the data. One idea for an architecture is to combine a simple Embedding + LSTM for the free text and concatenate the input with "Gene" and "Variation" Embeddings, leading into a final fully connected layer for the classes. Hopefully, this will capture the relationship between the text and the "Gene" + "Variation" columns.

* Train Word2Vec on a bigger corpus of genetic and medical data. Since Word2Vec is unsupervised, we can get better embeddings with more data, and consequently, better predictions

* Don't forget to do hyperparameter optimization when you're happy with a set of features. Stacked ensembling is also an almost guaranteed way to get a small boost to your score. We skipped this entirely in this notebook as this is usually the last step in the process. Try http://xcessiv.readthedocs.io/.

